# Real-time Gender Detection with OpenCV and TensorFlow/Keras

This code is a Python script for real-time gender detection using a pre-trained model and OpenCV.

In [1]:
'''
Importing required libraries and modules
TensorFlow for loading the pre-trained gender detection model, NumPy for numerical operations,
OpenCV for image processing, and the cvlib library for face detection.
'''

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

## Classifying only Gender

In [ ]:
'''load model using keras load_model function'''
model = load_model('gender_classification_model_v22.model')

'''Opening a connection to the default webcam (assuming it's the first camera, indicated by 0).
The cv2.VideoCapture function is used from the OpenCV library.
'''
webcam = cv2.VideoCapture(0)

classes = ['man','woman']    # defining list of classes for gender label

while webcam.isOpened():     # Starting an infinite loop to continuously capture frames from the webcam until the user decides to exit.

    # read a frame from webcam 
    status, frame = webcam.read()

    # apply face detection using cvlib `face` contains the coordinates of the detected face and `confidence` contains the confidence scores
    face, confidence = cv.detect_face(frame)

    # loop through detected face/faces
    for idx, f in enumerate(face):

        # get corner points (top-left and bottom-right) of face rectangle        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw green rectangle around the detected face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region from the frame
        face_crop = np.copy(frame[startY:endY,startX:endX])

        # if the croped face region is too small, skip further processing
        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        '''
        Preprocess the cropped face for gender detection:
        resize to (96, 96), normalize pixel values, convert to array, and add an extra dimension for the batch.
        '''
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # applying gender detection on face using the loaded model
        conf = model.predict(face_crop)[0] # model.predict return a 2D matrix, ex: [[9.9993384e-01 7.4850512e-05]]

        # geting the predicted gender label and confidence score
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        # determining the Y-coordinate for placing the label text above the face rectangle
        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release webCam and close all OpenCV windows when the program is finished
webcam.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 41ms/step


## Classifying Gender and Top wear color

In [7]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv

gender_model = load_model('gender_detection_largeDataset_v2.model')

webcam = cv2.VideoCapture(0)

while webcam.isOpened():

    status, frame = webcam.read()

    faces, confidences = cv.detect_face(frame)

    for idx, face in enumerate(faces):

        (startX, startY) = face[0], face[1]
        (endX, endY) = face[2], face[3]

        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        face_crop = np.copy(frame[startY:endY, startX:endX])

       # crop a region just below the detected face (adjusting it lower and wider)
        dress_region = frame[endY + 100:endY + 250, startX:endX]

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        face_crop = cv2.resize(face_crop, (96, 96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        gender_conf = gender_model.predict(face_crop)[0]
        gender_label = "Woman" if gender_conf[1] > gender_conf[0] else "Man"
        gender_percentage = max(gender_conf) * 100

         # calculate the color histogram in the dress region
        '''
        The color histogram of the dress_region is calculated using OpenCV's calcHist function.
        The dominant color is determined by finding the bin with the highest count in the histogram.
        The dominant color is represented in the BGR format.
        '''
        hist = cv2.calcHist([dress_region], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        dominant_color_bin = np.unravel_index(hist.argmax(), hist.shape)
        dominant_color_bgr = [int(b * 32) for b in dominant_color_bin]

        # Display the gender and dress color with percentage and color name
        gender_text = f"Gender: {gender_label} ({gender_percentage:.2f}%)"
        color_text = f"Dress Color: {dominant_color_bgr}"

        cv2.putText(frame, gender_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, color_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, dominant_color_bgr, 2)

        # Draw a rectangle around the adjusted dress color region
        cv2.rectangle(frame, (startX, endY + 100), (endX, endY + 250), dominant_color_bgr, 2)

    # Display output
    cv2.imshow("Gender and Dress Color Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 43ms/step


In [3]:
import cv2

def list_available_cameras():
    # Get the list of available camera indexes
    index = 0
    available_cameras = []
    
    while True:
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            break
        else:
            available_cameras.append(index)
        cap.release()
        index += 1

    return available_cameras

# List available cameras
cameras = list_available_cameras()

if not cameras:
    print("No cameras found.")
else:
    print("Available cameras:", cameras)


Available cameras: [0, 1]
